# word2vec

In [1]:
import pandas as pd
import numpy as  np
import jieba

In [2]:
train_df=pd.read_csv(r'H:\比赛\景区口碑评价分值预测\train_first.csv',engine='python',encoding='utf-8')
predict_df=pd.read_csv(r'H:\比赛\景区口碑评价分值预测\predict_first.csv',engine='python',encoding='utf-8')
data_df=pd.concat([train_df,predict_df],ignore_index=True)

In [3]:
jieba.load_userdict(r'H:\比赛\newdirect.txt')
stopwords = [line.strip() for line in open('H:\比赛\stop_words_ch.txt', 'r').readlines()]  

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\kate\AppData\Local\Temp\jieba.cache
Loading model cost 0.932 seconds.
Prefix dict has been built succesfully.


In [4]:
def jieba_fenci(raw):#, stopwords_list):
    # 使用结巴分词把文件进行切分
    word_list = jieba.lcut(raw, cut_all=False)
    return word_list

In [5]:
lst=[]
for wordlist in data_df['Discuss']:
    word_list=jieba_fenci(wordlist)#,stopwords)
    lst.append(word_list)

In [8]:
data_df['Discuss']=lst

In [9]:
test=data_df.loc[data_df['Score'].isnull(),:]
train=data_df.loc[data_df['Score'].notnull(),:]
test=test.reset_index(drop=True)

In [10]:
len(test.loc[test['Discuss'].isnull()])

0

In [11]:
from gensim.models.word2vec import Word2Vec

i:\python\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [27]:
X_train=train['Discuss'].values
X_test=test['Discuss'].values


In [13]:
y_train=train['Score'].values
y_test=test['Discuss'].values

In [14]:
model =Word2Vec(X_train,size=128,window=5,min_count=5,workers=4)

In [116]:
for k, s in model.most_similar(positive=[u'苏轼'], topn=30):
    print (k, s)

i:\python\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


雷峰塔 0.6918540000915527
武汉 0.6626406908035278
大雁塔 0.6409344673156738
布达拉宫 0.6397948265075684
岳阳楼 0.6230247020721436
塔 0.6177655458450317
东方明珠 0.6124304533004761
故宫 0.6025645732879639
长江大桥 0.6012500524520874
紫禁城 0.6009164452552795
电视塔 0.5989401340484619
瘦西湖 0.5983830690383911
圆明园 0.5926504135131836
澳门 0.592522144317627
城楼 0.5915618538856506
原址 0.5913763046264648
灵隐寺 0.5890893936157227
外滩 0.5881687998771667
玉龙雪山 0.5711280107498169
午门 0.5650074481964111
扬州 0.5645002126693726
天安门城楼 0.5644516348838806
大三巴牌坊 0.5634273290634155
御花园 0.5603914856910706
蓬莱阁 0.55787193775177
无数次 0.5524880290031433
大三巴 0.5523862838745117
大水法 0.5509259700775146
城墙 0.5499249696731567
拉萨 0.5496540069580078
电视 0.5490392446517944
皇宫 0.5461573600769043
嘉峪关 0.5422903895378113
少林寺 0.5325631499290466
天下第一关 0.5315659046173096
飞来峰 0.5282102823257446
维修 0.5280061960220337
教科书 0.5279356241226196
西辞 0.5268107652664185
乔家大院 0.526540219783783
景山 0.5252872705459595
法门寺 0.5231231451034546
景山公园 0.5211014747619629
蛮近 0.519088506698608

下雪天 0.36617305874824524
俯瞰 0.3661143481731415
古城墙 0.3660511374473572
新景观 0.36597979068756104
杂志 0.3659111261367798
梦回 0.36584794521331787
我敢 0.36568576097488403
康熙 0.36568209528923035
南岳 0.3656492233276367
哈利 0.36539536714553833
慈恩寺 0.36531010270118713
俑坑 0.36525118350982666
东北角 0.3650989532470703
雁塔 0.3650701642036438
两次 0.3650478422641754
热播 0.3650478422641754
顺路来 0.36489415168762207
头部 0.36485522985458374
金台 0.36485183238983154
钓鱼台国宾馆 0.36477214097976685
这尊 0.36461034417152405
长安城 0.36453795433044434
虽云高 0.3644874095916748
二千多年 0.3644700050354004
它 0.3643520474433899
侧门 0.3643444776535034
二十多年 0.3643154799938202
八一湖 0.3642338514328003
登楼 0.3642042875289917
楼上 0.3641844391822815
昆明 0.3641663193702698
土地 0.3641035556793213
地之坛 0.3640032410621643
行宫 0.3636024594306946
美庐 0.3635806739330292
筹建 0.36355698108673096
桑干河 0.36341404914855957
长春 0.3632643222808838
两千多年 0.36314427852630615
浦东 0.3629424273967743
兴起 0.36293625831604004
鲁迅 0.3625810444355011
浦东新区 0.3625107407569885
文化景观 0.3624605

#  用nlp模型表达X
拿过来取平均值

In [15]:
vocab=model.wv.vocab

In [16]:
def get_vector(word_list):
    res =np.zeros([128])
    count=0
    for word in word_list:
        if word in vocab:
            res +=model[word]
            count+=1
    return res/count

In [17]:
get_vector(['感觉','一般般'])

i:\python\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([ 0.75532045, -0.81370252,  0.26193773, -0.96625344, -0.78836603,
       -0.32833001,  0.6042702 , -0.57356894,  0.16929173, -0.08254247,
        0.78719386,  0.88176405, -0.18227625,  1.51845089,  0.35832953,
       -0.50999558, -0.22251597,  0.39357024,  0.04923624, -0.40696551,
       -0.06843164, -0.29943321,  0.3490098 , -0.43723235, -0.38013734,
        0.89748684,  1.66088301, -0.17637743,  0.18091852,  0.26740125,
       -0.04100943, -0.26510379, -0.77962632, -0.12828755, -1.66529983,
       -0.8724747 ,  1.82787579,  1.18531594, -0.47559637,  0.39401491,
       -0.202703  , -0.5625339 , -1.05461106,  0.76970161,  0.53027152,
       -1.49081305, -0.53381353,  0.16603685, -0.67120735, -0.13491718,
        0.66030017, -0.14897484, -0.25546071,  0.95490897, -1.47556394,
       -0.38028288,  0.22994944, -0.75874101,  0.90340608, -1.09319046,
        0.18074271, -1.05855474,  0.5895592 , -0.04760316, -0.14876411,
        0.28007428,  0.51041961,  0.86185788,  0.12932656,  0.06

In [28]:
X_train =[get_vector(x) for x in X_train]
X_test =[get_vector(x) for x in X_test]

i:\python\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
i:\python\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


In [19]:
X_train[1]

array([ 0.27431013,  0.62542599, -0.0760659 ,  0.33581011,  0.11985562,
        0.1964446 , -0.06115225,  0.02026551,  0.15392215,  0.23020705,
       -0.19312292,  0.03045941,  0.20908429, -0.07367652,  0.16572844,
       -0.62148322, -0.42766221,  0.14838217, -0.03737981, -0.02766087,
       -0.20624777, -0.20611153, -0.27013315, -0.29783322,  0.04761355,
        0.55956334,  0.37477226,  0.04569843, -0.72611649,  0.08473832,
       -0.07136598, -0.09615162,  0.14686223, -0.26375693, -0.46890996,
        0.03450114, -0.08713735, -0.03497128, -0.2369791 , -0.24065664,
       -0.4492933 , -0.46252431,  0.39360875,  0.6787124 , -0.30370745,
       -0.12319346,  0.16327469,  0.12074759, -0.37905886, -0.33605965,
       -0.58553416, -0.12997952,  0.60164359, -0.19572019, -0.16147663,
        0.00555497,  0.30714589, -0.16698052,  0.76225202, -0.03350551,
        0.80871635, -0.78376836,  0.45597048,  0.01562924,  0.82584848,
        0.14161559,  0.48743713, -0.36820989, -0.26939069, -0.05

# 建立ml模型

In [67]:
df=pd.DataFrame(X_train)
df['Score']=y_train
df2=df.loc[pd.notnull(df[1]),:].reset_index(drop=True)
y_train=df2['Score']
del(df2['Score'])
X_train=df2

In [82]:
from sklearn.linear_model import Ridge
from  sklearn.model_selection import cross_val_score

params =[0.1,0.5,1,3,5,7,10,12,16,20,25,30,35,40]
test_scores=[]
for param in params:
    clf =Ridge(param)
    test_score =1/(1+np.sqrt(-cross_val_score(clf,X_train,y_train,cv=10,scoring='mean_squared_error')))
    test_scores.append(np.mean(test_score)) 

i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring metho

i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring metho

i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring metho

i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring metho

In [83]:
test_scores

[0.5872912332259934,
 0.587292252396793,
 0.5872928103914357,
 0.587289847289691,
 0.5872825057387726,
 0.5872734152126564,
 0.5872588465398005,
 0.5872491829582047,
 0.5872307399370204,
 0.587213768488459,
 0.5871945859913824,
 0.587177391594486,
 0.5871618777656948,
 0.587147778209295]

In [ ]:
from sklearn.svm import SVR
from  sklearn.model_selection import cross_val_score

params =[0.1,0.5,1,3,5,7,10,12,16,20,25,30,35,40]
test_scores=[]
for param in params:
    clf =SVR(gamma=param)
    test_score =1/(1+np.sqrt(-cross_val_score(clf,X_train,y_train,cv=10,scoring='mean_squared_error')))
    test_scores.append(np.mean(test_score)) 
    print(param)

In [ ]:
import xgboost as xgb
from  sklearn.model_selection import cross_val_score

params =[3,5,7,10,12,16,20]
test_scores=[]
for param in params:
    clf =xgb(max_depth=params)
    test_score =1/(1+np.sqrt(-cross_val_score(clf,X_train,y_train,cv=10,scoring='mean_squared_error')))
    test_scores.append(np.mean(test_score)) 
    print(param)

In [ ]:
# 用CNN提升逼格
Vector表示出一个大的matrix
用CNN做“降维+注意力”
